# Performing parameter scans using UETOOLS

One typical application of UEDGE is to perform a sensitivity analysis of the solution by scanning different UEDGE variables, which can later be analyzed, as explained in Chapter 6. This Chapter will discuss some of the ways to automatically perform automated parameter scans in UEDGE.

## TOC:
* [Parameter scans using the time-dependent solver](#dt-scan)
* [Parameter scans using the continuation solver](#continuation-scan)


In [1]:
# Restore the case
from os import chdir
chdir('testcase_lores')
from uetools import Case
c = Case('input.yaml') # Create a test at variable "c" from the YAML input file

UEDGE configuration file /Users/holm10/.uetoolsrc read.
UETOOLS-style save successfully restored from /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_lores/nc20.hdf5

*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 1.54e+00



## Parameter scans using the time-dependent solver<a class="anchor" id="dt-scan"></a>

The most straightforward way to perform a parameter scan is to run a series of consequtive time-dependent runs, where the variable being scanned is changed in equidistant steps. The function Case.conv_step can be used for automating this task, whenever you need to scan a single variable:

In [2]:
help(c.solver.conv_step)

Help on method conv_step in module uetools.UeSolver.Solver:

conv_step(increment, name, var, ivar=None, stop=None, inversestep=False, **kwargs) method of uetools.UeSolver.Solver.Solver instance
    Converges var in steps of increment size using dt-solver
    
    **kwargs passed UeRun.converge
    
    Arguments
    ---------
    increment - float of increments to vary var with (var+increment)
    name - name of the case to be written, appended with var value
    var - str of var to be edited
    
    Keyword arguments
    -----------------
    ivar : int (default = None)
        index of var to be varied. Applies to whole array if None
    stop : float (default = None)
        value to stop scanning var at. if none, scans to var/5 or
        var*5, depending on direction
    inversestep : bool (default = False)
        if true, takes increments 1/var instead of var
    
    Returns
    -------
    None



Case.conv_step requires the increment of changed, a name for the the run series that is used to identify the resulting saves, and the parameter name for the variable to be scanned. In case of a multi-dimensional array the kwarg ivar, which is the variable index to be modified. The kwarg "stop" is used to define the stopping point of the simulation. The boolean kwarg "inversestep" is a special case used when converging the magnetic drift contribution, as the drift strength is defined as 1/b0. Setting inversestep=True takes steps in inverse parameter space, e.g. the steps are equidistant in 1/var rather than var. Additional kwargs are passed to the time-dependent solver.

There are a number of pre-defined conv_step derivatives, that can be used to scan one of the most common variabels, ncore, igaso, and b0.

In [3]:
for func in dir(c.solver):
    if 'conv_step_' in func: print(func)

conv_step_b0
conv_step_igaso
conv_step_ncore


Here, we demonstrate a short parameter scan of the core density, which is the core boundary condition in the example simulation:

In [4]:
c.solver.conv_step_ncore(-1e18, 'ncore_down', iisp=0, stop=15e18, dtreal=1e-6)

Solving for ncore[0]=1.90E+19
 Reading grid data from .../testcase_lores/gridue.hdf5.hdf5
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 Grid data read successfully:
     file name:   .../testcase_lores/gridue.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
  Updating Jacobian, npe =                      1
 iter=    0 fnrm=     0.1414213847911110     nfe=      1
  Updating Jacobian, npe =                      2
 iter=    1 fnrm=     0.1798415728504007E-02 nfe=      3
 iter=    2 fnrm=     0.3828659682065274E-06 nfe=      6


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Requested save-path ../solutions not found, writing to cwd!

 Initial successful time-step exists 

---------------------------------------------------------------
*** Number 

Intermediate solution written to ./ncore_down_ncore=1.900e+19_last_ii2.hdf5
 Total time = 1.2800E-05; Timestep = 3.4000E-06 

 Inner iteration #3 

 Reading grid data from .../testcase_lores/gridue.hdf5.hdf5
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 Grid data read successfully:
     file name:   .../testcase_lores/gridue.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
 iter=    0 fnrm=     0.5945577328683688E-01 nfe=      1
 iter=    1 fnrm=     0.4950455880877378E-04 nfe=      4
 iter=    2 fnrm=     0.2138212237260394E-09 nfe=      9


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down_ncore=1.900e+19_last_ii2.hdf5
 Total time = 1.6200E-05; Timestep = 3.4000E-06 

 Inner iteration #4 

 Reading 

 iter=    0 fnrm=     0.3853038640848055E-01 nfe=      1
  Updating Jacobian, npe =                      2
 iter=    1 fnrm=     0.4597343300510683E-03 nfe=      4
 iter=    2 fnrm=     0.1039158642178323E-07 nfe=      8


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down_ncore=1.900e+19_last_ii2.hdf5
 Inner iteration #1 

 Reading grid data from .../testcase_lores/gridue.hdf5.hdf5
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 Grid data read successfully:
     file name:   .../testcase_lores/gridue.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
 iter=    0 fnrm=     0.4334750459639705E-01 nfe=      1
 iter=    1 fnrm=     0.5784037443658204E-03 nfe=      4
 iter=    2 fnrm=     0.1691916613667305E-0

Intermediate solution written to ./ncore_down_ncore=1.900e+19_last_ii2.hdf5
 Total time = 8.6612E-04; Timestep = 1.3363E-04 

 Inner iteration #5 

 Reading grid data from .../testcase_lores/gridue.hdf5.hdf5
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 Grid data read successfully:
     file name:   .../testcase_lores/gridue.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
 iter=    0 fnrm=     0.6748607756357708E-01 nfe=      1
 iter=    1 fnrm=     0.1696904518401863E-03 nfe=      9
 iter=    2 fnrm=     0.2601180255480682E-08 nfe=     23


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down_ncore=1.900e+19_last_ii2.hdf5
 Total time = 9.9975E-04; Timestep = 1.3363E-04 

-------------------------------

Intermediate solution written to ./ncore_down_ncore=1.900e+19_last_ii2.hdf5
 Total time = 6.9491E-03; Timestep = 1.5448E-03 

 Inner iteration #3 

 Reading grid data from .../testcase_lores/gridue.hdf5.hdf5
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 Grid data read successfully:
     file name:   .../testcase_lores/gridue.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
 iter=    0 fnrm=     0.7922812441204902E-02 nfe=      1
 iter=    1 fnrm=     0.1430880710546666E-02 nfe=     10
 iter=    2 fnrm=     0.3214735930411870E-06 nfe=     19


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down_ncore=1.900e+19_last_ii2.hdf5
 Total time = 8.4939E-03; Timestep = 1.5448E-03 

 Inner iteration #4 

 Reading 

...

Intermediate solution written to ./ncore_down_ncore=1.400e+19_last_ii2.hdf5
 Total time = 6.8735E-04; Timestep = 1.9072E-05 

 Inner iteration #5 

 Reading grid data from .../testcase_lores/gridue.hdf5.hdf5
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 Grid data read successfully:
     file name:   .../testcase_lores/gridue.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
 iter=    0 fnrm=     0.1202949973772434     nfe=      1
 iter=    1 fnrm=     0.7581300915130423E-04 nfe=      7
 iter=    2 fnrm=     0.2460699349443414E-09 nfe=     21


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down_ncore=1.400e+19_last_ii2.hdf5
 Total time = 7.0642E-04; Timestep = 1.9072E-05 

-------------------------------

Intermediate solution written to ./ncore_down_ncore=1.400e+19_last_ii2.hdf5
 Total time = 1.5555E-03; Timestep = 2.2047E-04 

 Inner iteration #3 

 Reading grid data from .../testcase_lores/gridue.hdf5.hdf5
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 Grid data read successfully:
     file name:   .../testcase_lores/gridue.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
 iter=    0 fnrm=     0.8402159009666686E-01 nfe=      1
 iter=    1 fnrm=     0.9396254808006679E-03 nfe=     11
 iter=    2 fnrm=     0.6370053567829414E-04 nfe=     27
 iter=    3 fnrm=     0.2118318283415590E-09 nfe=     42


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down_ncore=1.400e+19_last_ii2.hdf5
 Total time = 1.7760E-03

 iter=    0 fnrm=     0.1378049025373686E-01 nfe=      1
 iter=    1 fnrm=     0.2039891553198475E-02 nfe=     11
 iter=    2 fnrm=     0.7146105284542545E-06 nfe=     21


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down_ncore=1.400e+19_last_ii2.hdf5
 Inner iteration #1 

 Reading grid data from .../testcase_lores/gridue.hdf5.hdf5
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 Grid data read successfully:
     file name:   .../testcase_lores/gridue.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
 iter=    0 fnrm=     0.8869218937900705E-02 nfe=      1
 iter=    1 fnrm=     0.1992422830536730E-02 nfe=     12
 iter=    2 fnrm=     0.1969806283610429E-05 nfe=     22


 nksol ---  iterm = 1.
           

Intermediate solution written to ./ncore_down_ncore=1.400e+19_last_ii2.hdf5
 Total time = 5.6888E-02; Timestep = 8.6653E-03 

 Inner iteration #5 

 Reading grid data from .../testcase_lores/gridue.hdf5.hdf5
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 Grid data read successfully:
     file name:   .../testcase_lores/gridue.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
 iter=    0 fnrm=     0.3494750491861740E-03 nfe=      1
 iter=    1 fnrm=     0.2477582692755700E-05 nfe=     12


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down_ncore=1.400e+19_last_ii2.hdf5
 Total time = 6.5553E-02; Timestep = 8.6653E-03 

----------------------------------------------------------------
*** Number of time-step

 iter=    1 fnrm=     0.2296518649919991E-11 nfe=     18


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down_ncore=1.400e+19_last_ii2.hdf5
 Total time = 5.5150E-01; Timestep = 1.0017E-01 

 Inner iteration #4 

 Reading grid data from .../testcase_lores/gridue.hdf5.hdf5
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 Grid data read successfully:
     file name:   .../testcase_lores/gridue.hdf5.hdf5
     run-ID:      EFITD    05/08/2002    #110465  3500ms                      
 iter=    0 fnrm=     0.6880620993644969E-08 nfe=      1
 iter=    1 fnrm=     0.4936381710220256E-12 nfe=     18


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            appr

Once ncore has reached the defined stopping threshold, a collection of saves have been created, spanning the parameter scan, which can be utilized in databases as discussed in Chapter 6.

In [5]:
!ls -h ncore_down*.hdf5

ncore_down_ncore=1.400e+19_last_ii2.hdf5
ncore_down_ncore=1.400e+19_last_ii2_UeDB.hdf5
ncore_down_ncore=1.500e+19_last_ii2.hdf5
ncore_down_ncore=1.600e+19_last_ii2.hdf5
ncore_down_ncore=1.700e+19_last_ii2.hdf5
ncore_down_ncore=1.800e+19_last_ii2.hdf5
ncore_down_ncore=1.900e+19_last_ii2.hdf5
ncore_down_ncore=8.000e+18_last_ii2.hdf5
ncore_down_ncore=9.000e+18_last_ii2.hdf5


## Parameter scans using the continuation solver<a class="anchor" id="continuation-scan"></a>
As mentioned in Chapter 4, the continuation solver is an ideal tool for performing parameter scans, as each successful solver step produces a converged solution, contrary to the time-dependent solver. Additionally, the continuation solver can perform a parameter scan in several dimensions. Note that each of the varied variables are evolved linearly and in an equal fraction of the difference from the starting point to the target. 

The frequency with which saves are written to the run-folder by the continuation solver is controlled by the variable saveres. Saveres defaults to 7, which means every 7th successful solver step is stored. The variable increments can, however, not be controlled in the continuation solver, as the algorithm tries to maximize the increments between solutions. 

In [6]:
c.solver.continuation_solve('ncore', target=10e18, savedir='ncore_down', index=0)


 Initial successful time-step exists 

===== MAIN LOOP 1/7 =====
    -Variable(s) being solved:
                                        - ncore
    -Progress                             0.000%
    -Advancing by                         0.100%
    -Steps to target at current delta:    1000

 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 iter=    0 fnrm=     0.8081220297313908E-03 nfe=      1
 iter=    1 fnrm=     0.2078379711127023E-05 nfe=     11
 iter=    2 fnrm=     0.5764511518124598E-11 nfe=     24


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ncore_down/progress0p100.hdf5

===== SUCCESS: ADVANCING VARIABLE =====
===== MAIN LOOP 2/7 =====
    -Variable(s) being solved:
                                        - ncore
    -Progress               


===== SUCCESS: ADVANCING VARIABLE ===== iter=    0 fnrm=     0.3940313045605055E-02 nfe=      1

===== MAIN LOOP 3/7 =====
    -Variable(s) being solved:
                                        - ncore
    -Progress                             3.208%
    -Advancing by                         0.484%
    -Steps to target at current delta:    199

 iter=    1 fnrm=     0.8141187182859252E-04 nfe=     10
 iter=    2 fnrm=     0.1863903868042748E-07 nfe=     19


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 iter=    0 fnrm=     0.3940735091960834E-02 nfe=      1
 iter=    1 fnrm=     0.8895756451431184E-04 nfe=     10
 iter=    2 fnrm=     0.2390238995434555E-07 nfe=     19


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, w

 Interpolants created; mype =                   -1
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    

===== FAIL 1/3 =====
===== DECREASE DELTA AND TRY AGAIN =====
===== MAIN LOOP 1/7 =====
    -Variable(s) being solved:
                                        - ncore
    -Progress                             13.082%
    -Advancing by                         1.171%
    -Steps to target at current delta:    74

  Updating Jacobian, npe =                      1
 iter=    0 fnrm=     0.1967029460770697E-01 nfe=      1
  Updating Jacobian, npe =                      2
 iter=    1 fnrm=     0.8957975979692585E-01 nfe=     11
 iter=    2 fnrm=     0.1580367063712674     nfe=     16
 iter=    3 fnrm=     0.7041321454268258E-01 nfe=     20
 iter=    4 fnrm=     0.5807161972075420E-01 nfe=     24
 iter=    5 fnrm=     0.8317704656992085     nfe=     28


 nksol ---  iterm = 4.
            the maximum allowable number of nonlinear
            iterations has been reached.


Intermediate solution written to ./ncore_down/progress15p082_dtrun_last_ii2.hdf5
 Total time = 4.0000E-05; Timestep = 1.0000E-05 

 Inner iteration #5 

Solving for delta=15.082%
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 iter=    0 fnrm=     0.4615683730946171E-02 nfe=      1
 iter=    1 fnrm=     0.2408460505054008E-05 nfe=      4


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down/progress15p082_dtrun_last_ii2.hdf5
 Total time = 5.0000E-05; Timestep = 1.0000E-05 

---------------------------------------------------------------
*** Number of time-step changes = 2 New time-step: 3.40E-05 ***
---------------------------------------------------------------
Solving for delta=15.082%
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IO

Intermediate solution written to ./ncore_down/progress15p082_dtrun_last_ii2.hdf5
 Total time = 1.3556E-04; Timestep = 3.7778E-06 

 Inner iteration #3 

Solving for delta=15.082%
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 iter=    0 fnrm=     0.5860644649255058E-02 nfe=      1
 iter=    1 fnrm=     0.4086710295165895E-03 nfe=     17
 iter=    2 fnrm=     0.5986953080741236E-04 nfe=     38
 iter=    3 fnrm=     0.2351906940269955E-05 nfe=     61


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down/progress15p082_dtrun_last_ii2.hdf5
 Total time = 1.3933E-04; Timestep = 3.7778E-06 

 Inner iteration #4 

Solving for delta=15.082%
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 Total time = 1.3933E-04; Timestep = 3.

...

Intermediate solution written to ./ncore_down/progress99p292_dtrun_last_ii2.hdf5
 Inner iteration #1 

Solving for delta=99.292%
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 iter=    0 fnrm=     0.7394969623359182E-01 nfe=      1
 iter=    1 fnrm=     0.9256445658398196E-02 nfe=      7
 iter=    2 fnrm=     0.6833674564667775E-04 nfe=     13
 iter=    3 fnrm=     0.3572040438055906E-08 nfe=     22


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down/progress99p292_dtrun_last_ii2.hdf5
 Total time = 1.8844E-04; Timestep = 3.7778E-06 

 Inner iteration #2 

Solving for delta=99.292%
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 iter=    0 fnrm=     0.4055960623996886E-01 nfe=      1
 iter=    1 fnrm=     0.11624932

 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 iter=    0 fnrm=     0.1367059869544656E-01 nfe=      1
 iter=    1 fnrm=     0.9325659229391461E-05 nfe=      7


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down/progress99p292_dtrun_last_ii2.hdf5
 Total time = 3.7174E-04; Timestep = 4.3671E-05 

 Inner iteration #3 

Solving for delta=99.292%
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 iter=    0 fnrm=     0.1400742014616006E-01 nfe=      1
 iter=    1 fnrm=     0.7565400001752984E-05 nfe=      7


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =             

Intermediate solution written to ./ncore_down/progress99p292_dtrun_last_ii2.hdf5
 Total time = 2.9518E-03; Timestep = 5.0484E-04 

 Inner iteration #4 

Solving for delta=99.292%
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 iter=    0 fnrm=     0.2266083052949559E-02 nfe=      1
 iter=    1 fnrm=     0.2625257920822112E-05 nfe=     11


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down/progress99p292_dtrun_last_ii2.hdf5
 Total time = 3.4567E-03; Timestep = 5.0484E-04 

 Inner iteration #5 

Solving for delta=99.292%
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 iter=    0 fnrm=     0.1800543699076640E-02 nfe=      1
 iter=    1 fnrm=     0.2045134517904676E-05 nfe=     11


 nksol ---  iterm = 1.
            ma

Intermediate solution written to ./ncore_down/progress99p292_dtrun_last_ii2.hdf5
 Total time = 4.3945E-02; Timestep = 5.8359E-03 

----------------------------------------------------------------
*** Number of time-step changes = 13 New time-step: 1.98E-02 ***
----------------------------------------------------------------
Solving for delta=99.292%
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
  Updating Jacobian, npe =                      1
 iter=    0 fnrm=     0.2692935736840042E-04 nfe=      1
  Updating Jacobian, npe =                      2
 iter=    1 fnrm=     0.4721287400411882E-07 nfe=     15


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
Intermediate solution written to ./ncore_down/progress99p292_dtrun_last_ii2.hdf5
 Inner iteration #1 

Solving for delta=99.292%
 TAR


===== STATIC ITERATION FOR DTREAL=2.50e-02 AT 100.0% =====
 iter=    0 fnrm=     0.4186995024253187E-04 nfe=      1
 iter=    1 fnrm=     0.8182391465355712E-07 nfe=     14
 iter=    2 fnrm=     0.1573194383969225E-08 nfe=     28
 iter=    3 fnrm=     0.1850242253369767E-12 nfe=     47


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.
 Interpolants created; mype =                   -1
 TARGET MATERIAL IS      GRAPHITE         
 BOMBARDING IONS ARE     D    
 iter=    0 fnrm=     0.3391556154412612E-04 nfe=      1
 iter=    1 fnrm=     0.1334170978371350E-06 nfe=     14
 iter=    2 fnrm=     0.2193138586970532E-08 nfe=     29
 iter=    3 fnrm=     0.2998001295561568E-12 nfe=     47


 nksol ---  iterm = 1.
            maxnorm(sf*f(u)) .le. ftol, where maxnorm() is
            the maximum norm function.  u is probably an
            approximate root of f.

===== 

The continuation solver will create a directory, as discussed in Chapter 4, containing a set of steady-state solutions which can be utilized by UETOOLS Database objects, as discussed in Chapter 6.

In [7]:
!ls ncore_down

SUCCESS.hdf5                       progress45p793.hdf5
progress0p100.hdf5                 progress88p042_dtrun_last_ii2.hdf5
progress100p000_staticiter.hdf5    progress89p601.hdf5
progress15p082_dtrun_last_ii2.hdf5 progress91p601_dtrun_last_ii2.hdf5
progress18p010.hdf5                progress94p719.hdf5
progress1p140.hdf5                 progress99p292_dtrun_last_ii2.hdf5
progress26p209.hdf5                progress9p887.hdf5
progress3p692.hdf5


As a first step to recovering from several consequtive failed solver steps, the continuation solver will perform static iterations at the value of the last successful execution to bring down the initial fnrm. These iterations, if successful, are named \*\_staticiter.hdf5 to identify these well-converged steady-state solutions. In some situations, this static iteration scheme will fail. In such circumstances, the continuation solver calls the time-dependent solver to ensure convergence. Such runs are identified by \*\_dtrun_last_ii2.hdf5 appearing in the name. All of these saves are compatible with Database Objects, as discussed Chapter 6.